# DQN playing basketball

In this notebook we show how to train a DQN agent to play [Atari DoubleDunk](https://gym.openai.com/envs/DoubleDunk-v0/), using screen pixels as state.

## Import libraries

In [ ]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torchvision.transforms as T
from gym import wrappers
from gym.wrappers.monitor import load_results
import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'running session on {device}')

In [ ]:
import sys
if not '..' in sys.path:
    sys.path.append('..')

from deeprl.agents import DQN, ReplayMemory
from deeprl.experiment import LearningParams, Experiment

## Define parameters

In [ ]:
AGENT_PARAMETERS = LearningParams({ 
                                    'BATCH_SIZE' : 128,
                                    'GAMMA' : 0.999,
                                    'EPS_START' : 0.9,
                                    'EPS_END' : 0.05,
                                    'EPS_DECAY' : 200,
                                    'TARGET_UPDATE' : 10
                                  })

MEMORY_CAPACITY = 10000
LOGGING_FOLDER = f'../logs/dqn-experiment-{datetime.datetime.now().strftime("%d-%m-%Y_%H:%M:%S")}'
NUM_EPISODES = 2

## Set up environment

In [ ]:
env = gym.make('DoubleDunk-v0')

# wrap environment for logging
env = wrappers.Monitor(env = env, 
                       directory = LOGGING_FOLDER, 
                       mode='training', 
                       force=True, 
                       write_upon_reset=False)

# REMARK: in order to save every episode video pass argument
#     video_callable=lambda episode_id: True
# see: https://github.com/openai/gym/issues/494

## Set up session

In [ ]:
# define replay memory
memory = ReplayMemory(capacity=MEMORY_CAPACITY)

In [ ]:
# image preprocessing
screen_transform = T.Compose([T.ToPILImage(),
                    T.Resize(84, interpolation=Image.CUBIC),
                    T.ToTensor()])

## Run experiment

In [ ]:
sess = Experiment(env = env, 
                  model_class = DQN,
                  params = AGENT_PARAMETERS,
                  device = device,
                  memory = memory,
                  screen_transform = screen_transform)

In [ ]:
sess.run(num_episodes=NUM_EPISODES)

### Visualize logged results

In [ ]:
results = load_results(LOGGING_FOLDER)
results